In [1]:
from ultralytics import YOLO

model = YOLO('yolo11s.pt')

100%|██████████| 18.4M/18.4M [00:01<00:00, 10.8MB/s]


In [2]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
results = model.train(data='config.yaml', epochs=20, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.78 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.76  Python-3.12.7 torch-2.7.0.dev20250127+cu126 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolo11s.pt, data=config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa

100%|██████████| 5.35M/5.35M [00:00<00:00, 9.28MB/s]


AMP: checks passed 


train: Scanning C:\Users\verdant\Desktop\basket-ai\dataset\train\labels.cache... 119 images, 0 backgrounds, 0 corrupt: 100%|██████████| 119/119 [00:00<?, ?it/s]
val: Scanning C:\Users\verdant\Desktop\basket-ai\dataset\train\labels.cache... 119 images, 0 backgrounds, 0 corrupt: 100%|██████████| 119/119 [00:00<?, ?it/s]


Plotting labels to C:\Users\verdant\Desktop\basket-ai\runs\detect\train13\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to C:\Users\verdant\Desktop\basket-ai\runs\detect\train13
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      4.48G      1.015     0.5938      1.067         60        640: 100%|██████████| 8/8 [00:04<00:00,  1.88it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.69it/s]

                   all        119       1277      0.887      0.848       0.89      0.586



20 epochs completed in 0.043 hours.
Optimizer stripped from C:\Users\verdant\Desktop\basket-ai\runs\detect\train13\weights\last.pt, 19.2MB
Optimizer stripped from C:\Users\verdant\Desktop\basket-ai\runs\detect\train13\weights\best.pt, 19.2MB

Validating C:\Users\verdant\Desktop\basket-ai\runs\detect\train13\weights\best.pt...
Ultralytics 8.3.76  Python-3.12.7 torch-2.7.0.dev20250127+cu126 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
YOLO11s summary (fused): 100 layers, 9,413,961 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  2.00it/s]


                   all        119       1277      0.877       0.85       0.89      0.586
                  ball        103        108      0.806       0.63      0.727      0.331
                 judge         78        173      0.917      0.959      0.964      0.695
                player        119        996      0.908      0.961      0.979      0.732
Speed: 0.2ms preprocess, 9.2ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to C:\Users\verdant\Desktop\basket-ai\runs\detect\train13
